Task 1: Read the data into RDD



In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkConf, SparkContext
import csv

In [ ]:
lines = sc.textFile("/content/EPL.csv")


In [ ]:
header = lines.first()
data = lines.filter(lambda row: row != header)

In [ ]:
def parse(line):
    return next(csv.reader([line]))

rdd = data.map(parse)

In [ ]:
print(rdd.take(5))

[['Sheffield United', 'Liverpool', '1.0', '1.0', 'D', '2006-2007'], ['Arsenal', 'Aston Villa', '1.0', '1.0', 'D', '2006-2007'], ['Everton', 'Watford', '2.0', '1.0', 'H', '2006-2007'], ['Newcastle United', 'Wigan Athletic', '2.0', '1.0', 'H', '2006-2007'], ['Portsmouth', 'Blackburn Rovers', '3.0', '0.0', 'H', '2006-2007']]


Task 2:Which season has the highest number of goals

In [ ]:
def is_float(x):
    try:
        float(x)
        return True
    except:
        return False

In [ ]:
season_goals = rdd.filter(lambda row: is_float(row[2]) and is_float(row[3])) \
                  .map(lambda row: (row[5], float(row[2]) + float(row[3]))) \
                  .reduceByKey(lambda x, y: x + y)


In [ ]:
highest_season = season_goals.takeOrdered(1, key=lambda x: -x[1])
print("Season with highest goals:", highest_season[0])

Season with highest goals: ('2011-2012', 1066.0)


Task 3:Season with lowest number of goals

In [ ]:
lowest_season = season_goals.takeOrdered(1, key=lambda x: x[1])
print("Season with lowest goals:", lowest_season[0])

Season with lowest goals: ('2006-2007', 931.0)


Task 4:Team with highest average goals per season

In [ ]:
team_season_goals = rdd.filter(lambda row: is_float(row[2]) and is_float(row[3])) \
    .flatMap(lambda row: [
        ((row[0], row[5]), float(row[2])),  # Home team goals
        ((row[1], row[5]), float(row[3]))   # Away team goals
    ]) \
    .reduceByKey(lambda x, y: x + y)

In [ ]:
team_goals_by_season = team_season_goals.map(lambda x: (x[0][0], (x[1], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [ ]:
team_avg_goals = team_goals_by_season.mapValues(lambda x: x[0] / x[1])

In [ ]:
best_team = team_avg_goals.takeOrdered(1, key=lambda x: -x[1])

In [ ]:
print("Team with highest average goals per season:", best_team[0])

Team with highest average goals per season: ('Manchester United', 72.25)


Task 5:Probabilities of Manchester United Wins, Draws, and Losses

In [ ]:
# Filter matches involving Manchester United
mu_matches = rdd.filter(lambda row: row[0] == "Manchester United" or row[1] == "Manchester United")

In [ ]:
# Map results: (result, 1)
mu_results = mu_matches.map(lambda row: (
    "Win" if (row[0] == "Manchester United" and row[4] == "H") or (row[1] == "Manchester United" and row[4] == "A")
    else "Loss" if (row[0] == "Manchester United" and row[4] == "A") or (row[1] == "Manchester United" and row[4] == "H")
    else "Draw",
    1
))


In [ ]:
# Count results
mu_counts = mu_results.reduceByKey(lambda x, y: x + y).collectAsMap()

In [ ]:
# Total matches
total_matches = sum(mu_counts.values())


In [ ]:
# Probabilities
p_win = mu_counts.get("Win", 0) / total_matches
p_loss = mu_counts.get("Loss", 0) / total_matches
p_draw = mu_counts.get("Draw", 0) / total_matches

print("P(Manchester United Wins):", p_win)
print("P(Manchester United Loses):", p_loss)
print("P(Manchester United Draws):", p_draw)

P(Manchester United Wins): 0.6359649122807017
P(Manchester United Loses): 0.16885964912280702
P(Manchester United Draws): 0.19517543859649122
